# Flexibility in a Search Model 

## Model

- homogeneous workers with utility $u(\cdot)$ if employed and flow (dis)utility $b$ if unemployed
- heterogeneous firms endowed with flexibility level $k \in \{0,1,..., K\}$ costing $c(k)$ with linear profit $y(x;k)-w(x;k)-c(k)$
- search parameters: discount rate $\rho$, unemployed meet firms at rate $\lambda$ (no on-the-job search), upon meeting draw match-specific productivity $x \sim G(x)$, bargaining parameter $\alpha$, employed face separation shock $\eta$


## Necessary Packages

In [ ]:
# General
import numpy as np
import pandas as pd 
import scipy.stats as stats

# Graphics
import matplotlib.pyplot as plt 
import seaborn as sns

# Estimation
from scipy.optimize import minimize

# Debugging
import pdb


## Data 
- Homogeneity measures: aged 25-55; at least college graduate; white; (maybe marital included, as it may be a concern for women?)
- employed workers earn wage $w_i$ at firm with flexibility level $k$
    - flexibility measures:
        - Schedule Flexibility 
            - 0: No flexibility in start and end times of work 
            - 1: Able to change start and end times of work 
        - Location Flexibility 
            - 0: Not able to work from home / unpaid to work from home 
            - 1:
        - Flexible Schedule Score
            - 0: No flexibility in start and end times of work 
            - 1: Informal policy allowing flexibility in start and end times of work
            - 2: Formal policy allowing for flexibility in start and end times of work
- unemployed workers have unemployment durations of $t_i$


In [ ]:
df=pd.read_stata('workfile.dta', columns=['sex','employed', 'flexsched', 'flex_sched_score', 'flexloc', 'flex_loc_score', 'hrwage', 'dur'])

### Men

In [ ]:
men = df[df['sex']=='male']
len(men)

In [ ]:
men['employed'].value_counts()

In [ ]:
men['hrwage'].describe()

In [ ]:
men['dur'].describe()

In [ ]:
fifth_pctl = np.zeros(1)

for k in range(3):
    tmp = men[men['flex_sched_score']==k]
    fifth = np.percentile(tmp['hrwage'],5)
    fifth_pctl = np.append(fifth_pctl, fifth)
#     print("5th percentile wage = " + str(fifth) + " for men with flex level " + str(k))
    
men['wage_trunc3']=men['hrwage']

for k in range(3):
    men['wage_trunc3'].iloc[(men['hrwage']<fifth_pctl[k+1]) & (men['flex_sched_score']==k)]=fifth_pctl[k+1] #k+1 because empty array initiates with zero

men['wage_trunc3'].groupby([men['employed'],men['flex_sched_score']]).describe()

In [ ]:
fifth_pctl = np.zeros(1)

for k in range(2):
    tmp = men[men['flexsched']==k]
    fifth = np.percentile(tmp['hrwage'],5)
    fifth_pctl = np.append(fifth_pctl, fifth)
#     print("5th percentile wage = " + str(fifth) + " for men with flex level " + str(k))
    
men['wage_trunc2']=men['hrwage']

for k in range(2):
    men['wage_trunc2'].iloc[(men['hrwage']<fifth_pctl[k+1]) & (men['flexsched']==k)]=fifth_pctl[k+1] #k+1 because empty array initiates with zero
        
men['wage_trunc2'].groupby([men['employed'],men['flexsched']]).describe()

In [ ]:
fifth_pctl = np.zeros(1)

for k in range(2):
    tmp = men[men['flexloc']==k]
    fifth = np.percentile(tmp['hrwage'],5)
    fifth_pctl = np.append(fifth_pctl, fifth)
#     print("5th percentile wage = " + str(fifth) + " for men with flex level " + str(k))
    
men['wage_trunc_loc_2']=men['hrwage']

for k in range(2):
    men['wage_trunc_loc_2'].iloc[(men['hrwage']<fifth_pctl[k+1]) & (men['flexloc']==k)]=fifth_pctl[k+1] #k+1 because empty array initiates with zero
        
men['wage_trunc_loc_2'].groupby([men['employed'],men['flexloc']]).describe()

### Women

In [ ]:
women = df[df['sex']=='female']
len(women)

In [ ]:
fifth_pctl = np.zeros(1)

for k in range(3):
    tmp = women[women['flex_sched_score']==k]
    fifth = np.percentile(tmp['hrwage'],5)
    fifth_pctl = np.append(fifth_pctl, fifth)
#     print("5th percentile wage = " + str(fifth) + " for women with flex level " + str(k))
    
women['wage_trunc3']=women['hrwage']

for k in range(3):
    women['wage_trunc3'].iloc[(women['hrwage']<fifth_pctl[k+1]) & (women['flex_sched_score']==k)]=fifth_pctl[k+1] #k+1 because empty array initiates with zero

women[['wage_trunc3','hrwage']].groupby([women['employed'],women['flex_sched_score']]).describe()    

In [ ]:
fifth_pctl = np.zeros(1)

for k in range(2):
    tmp = women[women['flexsched']==k]
    fifth = np.percentile(tmp['hrwage'],5)
    fifth_pctl = np.append(fifth_pctl, fifth)
#     print("5th percentile wage = " + str(fifth) + " for women with flex level " + str(k))
    
women['wage_trunc2']=women['hrwage']

for k in range(2):
    women['wage_trunc2'].iloc[(women['hrwage']<fifth_pctl[k+1]) & (women['flexsched']==k)]=fifth_pctl[k+1] #k+1 because empty array initiates with zero

women[['wage_trunc2','hrwage']].groupby([women['employed'],women['flexsched']]).describe()        

In [ ]:
fifth_pctl = np.zeros(1)

for k in range(2):
    tmp = women[women['flexloc']==k]
    fifth = np.percentile(tmp['hrwage'],5)
    fifth_pctl = np.append(fifth_pctl, fifth)
    print("5th percentile wage = " + str(fifth) + " for women with flex level " + str(k))
    
women['wage_trunc_loc_2']=women['hrwage']

for k in range(2):
    women['wage_trunc_loc_2'].iloc[(women['hrwage']<fifth_pctl[k+1]) & (women['flexloc']==k)]=fifth_pctl[k+1] #k+1 because empty array initiates with zero

women[['wage_trunc_loc_2','hrwage']].groupby([women['employed'],women['flexloc']]).describe()        

## Model Independent Functions

In [ ]:
def lognormpdf(x: np.array, μ: float, σ: float):
    """
    Calculates lognormal pdf without stats packages
    """
    
    denom = x * σ * np.sqrt(2*np.pi)
    exp_num = -(np.log(x)-μ)**2
    exp_denom = 2 * σ * σ
    num = np.exp(exp_num/exp_denom)
    
    return num/denom

In [ ]:
def lognormsf(x: np.array, μ: float, σ: float):
    """
    Calculates lognormal cdf with scipy.stats normal cdf
    """
    
    lnx = np.log(x)
    num = lnx - μ
    denom = σ
    
    return 1-stats.norm.cdf(num/denom)

In [ ]:
def bootstrap(data: pd.DataFrame, n_samples:int):
    """
    Thanks, Caleb
    """
    bootstrapped_sample_list = []
    
    for n in range(n_samples):
        nth_sample = data.sample(frac=1, replace=True)
        bootstrapped_sample_list.append(nth_sample)
    
    return bootstrapped_sample_list

In [ ]:
def std_error(values):
    """
    Calculates the standard error (standard deviation of values divided by square root of the number of values) of some values 
    """
    
    stderr = np.std(values) / np.sqrt(len(values))

    return stderr

In [ ]:
def fit_stats(values):
    """
    
    """
    
    mean = np.mean(values)
    
    stderr = std_error(values)
    
#     ci = np.percentile(values, [2.5,97.5])# 95% C.I.

    return print("Boostrapped value ", str(mean), "\nStandard error    ", str(stderr),"\n")#, "\n95% Confidence interval", str(ci)

## Utility $u(w,k; \gamma) = w(x,k) + \gamma k$ and Productivity assumption $y(x,k; \zeta) = \zeta kx$

- One issue: three variables for flexibility (cost $c$, productivity $\zeta$, and utility $\gamma$
    - Create iterative process where 2 out of 3 set by lit, find 3rd, use in next estimation to find other? 
        - Potentially issues in ordering? Fixed point argument? 
    - Remove cost $c$ by argument that TFP will account for the relative inputs and outputs ? (see notes on Bloom et al 2015)
    - Set two out of three using literature values (akin to Macro calibration)
    - 2-stage estimation? 
- Using Nelder-Mead, there is not a Hessian to calculate standard errors, so use bootstrap method 
    - Other idea: check log likelihood when only having one value to identify at a time
    

### Functions

In [ ]:
def Pr_wage_given_match(data: pd.DataFrame, flex: str, wage: str, res_wage: np.array, c_k: np.array, ζ: float, γ: float, α: float, μ: float, σ: float):
    """
    Calculates probability of a wage draw conditional on a match being formed 
    
    Inputs
    - data: DataFrame
    - flex: string for name of flexibility column
    - wage: string for name of wage column
    - res_wage: array of observed minimum wage of flexibility k
    - c_k: Kx1 array of cost of providing flexibility
    - ζ: productivity weight of flexibility k
    - γ: utility weight of flexibility k
    - α: bargaining parameter
    - μ: location parameter of the log-normal wage distribution
    - σ: scale parameter of the log-normal wage distribution
    
    Functions
    - lognormpdf(x: np.array, μ: float, σ: float)
    - lognormsf(x: np.array, μ: float, σ: float)
    """
    employed_indiv = np.zeros(1) #sets first entry to zero 

    for k in range(len(c_k)):
        tmp = data[data[flex]==k]
        g = ( 1/( α*ζ*(k+1) ) ) * lognormpdf(( tmp[wage] - (1-α)*( res_wage[k] - (γ*(k+1)) ) + α*c_k[k] )/( α*ζ*(k+1) ), μ, σ )
        G_tilde = lognormsf( ( res_wage[k] + c_k[k] - (γ*(k+1)) )/(ζ*(k+1)), μ, σ )
        divide_thing = g/G_tilde
        employed_indiv = np.append(employed_indiv, divide_thing)
    
    return employed_indiv[1:] #removes first entry 


In [ ]:
def hazard(res_wage: float, c_k: np.array, p_k: np.array, λ: float, ζ: float, γ: float, μ: float, σ: float):
    """
    Calculates the hazard rate out of employment 
    
    Inputs
    - res_wage: array of observed minimum wage of flexibility k
    - c_k: Kx1 array of cost of providing flexibility
    - p_k: Kx1 array of probability of each level of flexibility
    - λ: arrival rate of offer
    - ζ: productivity weight of flexibility k
    - γ: utility weight of flexibility k    
    - μ: location parameter of the log-normal wage distribution
    - σ: scale parameter of the log-normal wage distribution
    
    Functions
    - lognormsf(x: np.array, μ: float, σ: float)
    """
    
    prob_sum = 0
    
    if len(p_k)!=len(c_k):
        return print("Length of p_k and c_k do not match.")
    else:
        for k in range(len(c_k)):
            prob_sum += p_k[k] * lognormsf( ( res_wage[k] + c_k[k] - (γ*(k+1)) )/(ζ*(k+1)), μ, σ ) #k+1 because Python index 0

    return λ*prob_sum

In [ ]:
def log_L(data: pd.DataFrame, flex: str, wage: str, dur: str, c_k: np.array, α: float, λ: float, η: float, ζ: float, γ: float, μ: float, σ: float):
    """
    
    Inputs
    - data: DataFrame of all individuals
    - flex: string for column of flexibility index (k)
    - wage: string for column of wage data 
    - dur: string for unemployment duration data
    - c_k: Kx1 array of cost of providing flexibility
    - α: bargaining parameter
    - λ: arrival rate of offer
    - η: termination rate
    - ζ: productivity weight of flexibility k
    - γ: utility weight of flexibility k   
    - μ: location parameter of the log-normal wage distribution
    - σ: scale parameter of the log-normal wage distribution
    
    Functions
    - hazard(res_wage: np.array, c_k: np.array, p_k: np.array, λ: float, ζ: float, γ: float, μ: float, σ: float)
    - Pr_wage_given_match(data: pd.DataFrame, flex: str, wage: str, res_wage: np.array, c_k: np.array, ζ: float, γ: float, α: float, μ: float, σ: float)
    """
    res_wage = data[wage].groupby(data[flex]).min().array
    p_k = data[flex].value_counts(normalize=True).sort_index().array
    
    N_log_h = data.count() * np.log( hazard(res_wage, c_k, p_k, λ, ζ, γ, μ, σ) )
    N_log_h_plus_η = data.count() * np.log( hazard(res_wage, c_k, p_k, λ, ζ, γ, μ, σ) + η )
    
    empl_data = np.sum( np.log( Pr_wage_given_match(data, flex, wage, res_wage, c_k, ζ, γ, α, μ, σ) ) )
    
    Nu_log_η = data[dur].count() * np.log(η)
    
    unempl_data = hazard(res_wage, c_k, p_k, λ, ζ, γ, μ, σ) * np.sum(data[dur])
    
    logL = -(N_log_h - N_log_h_plus_η + empl_data + Nu_log_η - unempl_data)
    
    return logL[0]

In [ ]:
def est_gamma(data: pd.DataFrame, flex: str, wage: str, duration: str):
    """
    Estimate parameter values for λ, η, γ, μ, σ
        * Currently hard-coded for binary flexibility measures in cost array! *
    """
    
    params = np.array([λ, η, γ, data[wage].mean(), data[wage].std()])
    
    Bounds = ((0,99), (0,99), (-99,99), (0,99), (0,99))
    
    logL_opt = lambda x: log_L(data, flex, wage, duration, 
                                np.zeros(2), α, x[0], x[1], ζ, x[2],
                                x[3], x[4])

    est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000}, bounds=Bounds)
    
    return [est.fun, est.x]

In [ ]:
def est_zeta(data: pd.DataFrame, flex: str, wage: str, duration: str):
    """
    Estimate parameter values for λ, η, ζ, μ, σ
        * Currently hard-coded for binary flexibility measures in cost array! *
    """
    
    params = np.array([λ, η, ζ, data[wage].mean(), data[wage].std()])
    
    Bounds = ((0,99), (0,99), (-99,99), (0,99), (0,99))
    
    logL_opt = lambda x: log_L(data, flex, wage, duration, 
                                np.zeros(2), α, x[0], x[1], x[2], γ,
                                x[3], x[4])

    est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000}, bounds=Bounds)
    
    return [est.fun, est.x]

In [ ]:
def bootstrap_est(data: pd.DataFrame, flex: str, wage: str, dur: str, n_samples:int):
    """
    
    * Very slow: 36.6 s ± 8.07 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
    """
    
    bootstrapped_data = bootstrap(data, n_samples)   
    
    # Gamma

    print("Gamma Estimation\n") 
    
    logL_gamma = []
    lambdas_gamma = []
    etas_gamma = []
    mus_gamma = []
    sigmas_gamma = []
    gammas = []

    for sample in bootstrapped_data:
        est_g = est_gamma(sample, 'flexsched', 'wage_trunc2', 'dur')

        logL_gamma.append(est_g[0])

        estimated = est_g[1]

        lambdas_gamma.append(estimated[0])
        etas_gamma.append(estimated[1])
        gammas.append(estimated[2])
        mus_gamma.append(estimated[3])
        sigmas_gamma.append(estimated[4])  
        
    gamma_estimates = [logL_gamma, lambdas_gamma, etas_gamma, mus_gamma, sigmas_gamma, gammas]
    
    gamma_stats = []
    for i in gamma_estimates:
        gamma_stats.append(fit_stats(i))
    
    # Zeta
    
    print("\nZeta Estimation\n")
    
    logL_zetas = []
    lambdas_zetas = []
    etas_zetas = []
    mus_zetas = []
    sigmas_zetas = []
    zetas = []

    for sample in bootstrapped_data:
        est_z = est_zeta(sample, 'flexsched', 'wage_trunc2', 'dur')

        logL_zetas.append(est_z[0])

        tmp = est_z[1]

        lambdas_zetas.append(tmp[0])
        etas_zetas.append(tmp[1])
        zetas.append(tmp[2])
        mus_zetas.append(tmp[3])
        sigmas_zetas.append(tmp[4])
        
    zeta_estimates = [logL_zetas, lambdas_zetas, etas_zetas, mus_zetas, sigmas_zetas, zetas]
    
    zeta_stats = []
    for i in zeta_estimates:
        zeta_stats.append(fit_stats(i))

#     return print("Gamma stats: ", gamma_stats,
#                 "\nZeta stats: ", zeta_stats)

In [ ]:
# Parameters to be estimated 

c_k = np.array([0,5,10])
λ = 1
η = 1
ζ = 0.25 # Estimate of TFP increase from Bloom et al (2015)
α = 0.5

### Men, Binary Schedule Flex

In [ ]:
γ = 0.6875 # Relative value of schedule flexibility to high salary from He et al (2021)

bootstrap_est(men, 'flexsched', 'wage_trunc2', 'dur', 10)

### Women, Binary Schedule Flex

In [ ]:
γ = 0.6875 # Relative value of schedule flexibility to high salary from He et al (2021)

bootstrap_est(women, 'flexsched', 'wage_trunc2', 'dur', 10)

### Men, Binary Location Flex

In [ ]:
γ = 0.625 # Relative value of location flexibility to high salary from He et al (2021)

bootstrap_est(men, 'flexloc', 'wage_trunc_loc_2', 'dur', 10)

### Women, Binary Location Flex

In [ ]:
γ = 0.625 # Relative value of location flexibility to high salary from He et al (2021)

bootstrap_est(women, 'flexloc', 'wage_trunc_loc_2', 'dur', 10)

### Estimation: Men, K=3

In [ ]:
men['flex_sched_score'].value_counts(normalize=True, sort=False)

In [ ]:
prob_k = np.array([0.274557, 0.547504, 0.177939])

In [ ]:
# Two-stage estimation

## Labor Market Variables in the first stage
Bounds1 = ((0,999), (0,999), (0,999), (0,999))

params1 = np.array([λ, η, men['wage_trunc3'].mean(), men['wage_trunc3'].std()])

logL_opt1 = lambda x: log_L(men, 'flex_sched_score', 'wage_trunc3', 'dur', men['wage_trunc3'].min(), 
                            np.array([0,7,10]), prob_k, α, x[0], x[1], ζ, γ,
                            x[2], x[3])

est4 = minimize(logL_opt1, params1, method='Nelder-Mead', options={'maxiter':500, 'disp':True}, bounds=Bounds1)

## Flexibility Variables in the second stage
params2 = np.array([c_k[1], c_k[2], ζ, γ])

logL_opt2 = lambda x: log_L(men, 'flex_sched_score', 'wage_trunc3', 'dur', men['wage_trunc3'].min(), 
                            np.array([0,x[0],x[1]]), prob_k, α, est4.x[0], est4.x[1], x[2], x[3],
                            est4.x[2], est4.x[3])

est4_second = minimize(logL_opt2, params2, method='Nelder-Mead', options={'maxiter':500, 'disp':True})

In [ ]:
print("Men's Labor market variables [λ, η, μ, σ] = "+ str(est4.x))
print("Men's Flexibility variables [c(1), c(2), ζ, γ] = "+ str(est4_second.x))

### Estimation: Women, K=3 
Estimation is finding $\mu=0$

In [ ]:
women['flex_sched_score'].value_counts(normalize=True, sort=False)

In [ ]:
prob_k = np.array([0.402640, 0.444719, 0.152640])

In [ ]:
# Two-stage estimation

# Labor Market Variables in the first stage
Bounds1 = ((0,999), (0,999), (0,999), (0,999))

params1 = np.array([λ, η, women['wage_trunc3'].mean(), women['wage_trunc3'].std()])

logL_opt1 = lambda x: log_L(women, 'flex_sched_score', 'wage_trunc3', 'dur', women['wage_trunc3'].min(), 
                            np.array([0,7,10]), prob_k, α, x[0], x[1], ζ, γ,
                            x[2], x[3])

est4 = minimize(logL_opt1, params1, method='Nelder-Mead', options={'maxiter':500, 'disp':True}, bounds=Bounds1)

# Flexibility Variables in the second stage
params2 = np.array([c_k[1], c_k[2], ζ, γ])

logL_opt2 = lambda x: log_L(women, 'flex_sched_score', 'wage_trunc3', 'dur', women['wage_trunc3'].min(), 
                            np.array([0,x[0],x[1]]), prob_k, α, est4.x[0], est4.x[1], x[2], x[3],
                            est4.x[2], est4.x[3])

est4_second = minimize(logL_opt2, params2, method='Nelder-Mead', options={'maxiter':1000, 'disp':True})

In [ ]:
print("Women's Labor market variables [λ, η, μ, σ] = "+ str(est4.x))
print("Women's Flexibility variables [c(1), c(2), ζ, γ] = "+ str(est4_second.x))

## Utility Linear in wage and Productivity assumption $y(x;k) = kx$

### Functions

In [ ]:
def Pr_wage_given_match(data: pd.DataFrame, flex: str, wage: str, res_wage: float, c_k: np.array, α: float, μ: float, σ: float):
    """
    Calculates probability of a wage draw conditional on a match being formed 
    
    Inputs
    - data: DataFrame
    - flex: string for name of flexibility column
    - wage: string for name of wage column
    - res_wage: float of observed minimum wage
    - c_k: Kx1 array of cost of providing flexibility
    - α: bargaining parameter
    - μ: location parameter of the log-normal wage distribution
    - σ: scale parameter of the log-normal wage distribution
    
    Functions
    - lognormpdf(x: np.array, μ: float, σ: float)
    - lognormsf(x: np.array, μ: float, σ: float)
    """
    employed_indiv = np.zeros(1) #sets first entry to zero 

    for k in range(len(c_k)):
        tmp = data[data[flex]==k]
        g = ( 1/( α*(k+1) ) ) * lognormpdf( ( 1/( α*(k+1) ) )*( tmp[wage] - (1-α)*res_wage + α*c_k[k] ), μ, σ )
        G_tilde = lognormsf( ( 1/(k+1) )*( res_wage + c_k[k] ), μ, σ )
        divide_thing = g/G_tilde
        employed_indiv = np.append(employed_indiv, divide_thing)
    
    return employed_indiv[1:] #removes first entry 

In [ ]:
def hazard(res_wage: float, c_k: np.array, p_k: np.array, λ: float, μ: float, σ: float):
    """
    Calculates the hazard rate out of employment 
    
    Inputs
    - res_wage: float of observed minimum wage
    - c_k: Kx1 array of cost of providing flexibility
    - p_k: Kx1 array of probability of each level of flexibility
    - λ: arrival rate of offer
    - μ: location parameter of the log-normal wage distribution
    - σ: scale parameter of the log-normal wage distribution
    
    Functions
    - lognormsf(x: np.array, μ: float, σ: float)
    """
    
    prob_sum = 0
    
    if len(p_k)!=len(c_k):
        return print("Length of p_k and c_k do not match.")
    else:
        for k in range(len(c_k)):
            prob_sum += p_k[k] * lognormsf( ( 1/(k+1) )*( res_wage + c_k[k]), μ, σ ) #k+1 because Python index 0

    return λ*prob_sum

In [ ]:
def log_L(data: pd.DataFrame, flex: str, wage: str, dur: str, res_wage: float, c_k: np.array, p_k: np.array, α: float, λ: float, η: float, μ: float, σ: float):
    """
    
    Inputs
    - data: DataFrame of all individuals
    - flex: string for column of flexibility index (k)
    - wage: string for column of wage data 
    - dur: string for unemployment duration data
    - res_wage: float of observed minimum wage
    - c_k: Kx1 array of cost of providing flexibility
    - p_k: Kx1 array of probability of each level of flexibility
    - α: bargaining parameter
    - λ: arrival rate of offer
    - η: termination rate
    - μ: location parameter of the log-normal wage distribution
    - σ: scale parameter of the log-normal wage distribution
    
    Functions
    - hazard(res_wage: np.array, c_k: np.array, p_k: np.array, λ: float, μ: float, σ: float)
    - Pr_wage_given_match(data: pd.DataFrame, flex: str, wage: str, res_wage: np.array, c_k: np.array,  α: float, μ: float, σ: float)
    """
    
    N_log_h = data.count() * np.log( hazard(res_wage, c_k, p_k, λ, μ, σ) )
    N_log_h_plus_η = data.count() * np.log( hazard(res_wage, c_k, p_k, λ, μ, σ) + η )
    
    empl_data = np.sum( np.log( Pr_wage_given_match(data, flex, wage, res_wage, c_k, α, μ, σ) ) )
    
    Nu_log_η = data[dur].count() * np.log(η)
    
    unempl_data = hazard(res_wage, c_k, p_k, λ, μ, σ) * np.sum(data[dur])
    
    logL = -(N_log_h - N_log_h_plus_η + empl_data + Nu_log_η - unempl_data)
    
    return logL[0]

In [ ]:
# Parameters to be estimated 

c_k = np.array([0,5,10])
λ = 10
η = 10

### Estimation: Men, K=3

In [ ]:
men['flex_sched_score'].value_counts(normalize=True, sort=False)

In [ ]:
prob_k = np.array([0.274557, 0.547504, 0.177939])

In [ ]:
params = np.array([c_k[1], c_k[2], λ, η, men['wage_trunc3'].mean(), men['wage_trunc3'].std()])

logL_opt = lambda x: log_L(men, 'flex_sched_score', 'wage_trunc3', 'dur', men['wage_trunc3'].min(), 
                            np.array([0,x[0],x[1]]), prob_k, 0.5, x[2], x[3],
                            x[4], x[5])

est4 = minimize(logL_opt, params, method='Nelder-Mead')

In [ ]:
est4.success

In [ ]:
est4.x

In [ ]:
est4.fun

In [ ]:
params = np.array([c_k[1], c_k[2], λ, η, men['hrwage'].mean(), men['hrwage'].std()])

logL_opt = lambda x: log_L(men, 'flex_sched_score', 'hrwage', 'dur', men['hrwage'].min(), 
                            np.array([0,x[0],x[1]]), prob_k, 0.5, x[2], x[3],
                            x[4], x[5])

est3 = minimize(logL_opt, params, method='Nelder-Mead')

In [ ]:
est3.success

In [ ]:
est3.x

In [ ]:
est3.fun

### Estimation: Men, K=2

In [ ]:
men['flex'].value_counts(normalize=True, sort=False)

In [ ]:
prob_k = np.array([0.274557, 0.725443])

In [ ]:
params = np.array([c_k[1], λ, η, men['wage_trunc2'].mean(), men['wage_trunc2'].std()])

logL_opt = lambda x: log_L(men, 'flex', 'wage_trunc2', 'dur', men['wage_trunc2'].min(), 
                            np.array([0,x[0]]), prob_k, 0.5, x[1], x[2],
                            x[3],x[4])

est2 = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':8000})

In [ ]:
est2.success

In [ ]:
est2.x

In [ ]:
est2.fun

In [ ]:
params = np.array([c_k[1], λ, η, men['hrwage'].mean(), men['hrwage'].std()])

logL_opt = lambda x: log_L(men, 'flex', 'hrwage', 'dur', men['hrwage'].min(), 
                            np.array([0,x[0]]), prob_k, 0.5, x[1], x[2],
                            x[3],x[4])

est1 = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':8000})

In [ ]:
est1.success

In [ ]:
est1.x

In [ ]:
est1.fun

### Estimation: Women, K=3

In [ ]:
women['flex_sched_score'].value_counts(normalize=True, sort=False)

In [ ]:
prob_k = np.array([0.402640, 0.444719, 0.152640])

In [ ]:
params = np.array([c_k[1], c_k[2], λ, η, women['wage_trunc3'].mean(), women['wage_trunc3'].std()])

logL_opt = lambda x: log_L(women, 'flex_sched_score', 'wage_trunc3', 'dur', women['wage_trunc3'].min(), 
                            np.array([0,x[0],x[1]]), prob_k, 0.5, x[2], x[3],
                            x[4], x[5])

est4 = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':8000})

c_1 = 5, c_2 = 10

In [ ]:
est4.success

In [ ]:
est4.x

In [ ]:
est4.fun

In [ ]:
# Changes every time I run it for some reason.

params = np.array([c_k[1], c_k[2], λ, η, women['hrwage'].mean(), women['hrwage'].std()])

logL_opt = lambda x: log_L(women, 'flex_sched_score', 'hrwage', 'dur', women['hrwage'].min(), 
                            np.array([0,x[0],x[1]]), prob_k, 0.5, x[2], x[3],
                            x[4], x[5])

est3 = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':8000})

In [ ]:
est3.success

In [ ]:
est3.x

In [ ]:
est3.fun

### Estimation: Women, K=2

In [ ]:
women['flex'].value_counts(normalize=True, sort=False)

In [ ]:
prob_k = np.array([0.40264, 0.59736])

In [ ]:
# Also changes every time I run it

params = np.array([c_k[1], λ, η, women['wage_trunc2'].mean(), women['wage_trunc2'].std()])

logL_opt = lambda x: log_L(women, 'flex', 'wage_trunc2', 'dur', women['wage_trunc2'].min(), 
                            np.array([0,x[0]]), prob_k, 0.5, x[1], x[2],
                            x[3],x[4])

est2 = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':8000})

In [ ]:
est2.success

In [ ]:
est2.x

In [ ]:
est2.fun

In [ ]:
# Consistent across runs

params = np.array([c_k[1], λ, η, women['hrwage'].mean(), women['hrwage'].std()])

logL_opt = lambda x: log_L(women, 'flex', 'hrwage', 'dur', women['hrwage'].min(), 
                            np.array([0,x[0]]), prob_k, 0.5, x[1], x[2],
                            x[3],x[4])

est1 = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':8000})

In [ ]:
est1.success

In [ ]:
est1.x

In [ ]:
est1.fun

# Figures

## Flex Schedule Score (k = 3)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(12, 8))

for k in range(3):
    tmp = df[(df['flex_sched_score']==k) & (df['sex']=='male') & (df['employed']==1)]
    sns.distplot(tmp['hrwage'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=100, ax=ax[k])
#     ax[k].legend(['Flexibility Level ' + str(k)])
    ax[k].set_ylim([0,0.1])
    ax[k].set_xlim([0,75])
    ax[k].set(xlabel = 'Hourly Wage for Men with Flexibile Schedule Score ' +str(k))

# ax.set(xlabel="Distribution of Men's Hourly Wage (raw)")

plt.tight_layout()

fig.savefig('./hrwage_men_3flex.png', bbox_inches='tight', transparent=True)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(12, 8))

for k in range(3):
    tmp = df[(df['flex_sched_score']==k) & (df['sex']=='female') & (df['employed']==1)]
    sns.distplot(tmp['hrwage'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=100, ax=ax[k])
#     ax[k].legend(['Flexibility Level ' + str(k)])
    ax[k].set_ylim([0,0.1])
    ax[k].set_xlim([0,75])
    ax[k].set(xlabel = 'Hourly Wage for Women with Flexibile Schedule Score ' +str(k))

# ax.set(xlabel="Distribution of Men's Hourly Wage (raw)")

plt.tight_layout()

fig.savefig('./hrwage_women_3flex.png', bbox_inches='tight', transparent=True)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(12, 8))

for k in range(3):
    tmp = women[(women['flex_sched_score']==k) & (women['employed']==1)]
    sns.distplot(tmp['wage_trunc3'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=100, ax=ax[k])
#     ax[k].legend(['Flex Level ' + str(k)])
    ax[k].set_ylim([0,0.1])
    ax[k].set_xlim([0,75])
    ax[k].set(xlabel = 'Truncated Hourly Wage for Women with Flexibile Schedule Score ' +str(k))

#ax.set(xlabel="Distribution of Men's Hourly Wage")

plt.tight_layout()

fig.savefig('./wageTrunc_women_3flex.png', bbox_inches='tight', transparent=True)

## Binary Flexibility Measure

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8))

for k in range(2):
    tmp = df[(df['flex']==k) & (df['sex']=='male') & (df['employed']==1)]
    sns.distplot(tmp['hrwage'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=100, ax=ax[k])
#     ax[k].legend(['Flexibility Level ' + str(k)])
    ax[k].set_ylim([0,0.1])
    ax[k].set_xlim([0,75])
    if k == 1:
        ax[k].set(xlabel = 'Hourly Wage for Men with Flexible Schedule')
    elif k == 0:
        ax[k].set(xlabel = 'Hourly Wage for Men without Flexible Schedule')
    else:
        print("Not binary k")

# ax.set(xlabel="Distribution of Men's Hourly Wage (raw)")

plt.tight_layout()

fig.savefig('./hrwage_men_2flex.png', bbox_inches='tight', transparent=True)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8))

for k in range(2):
    tmp = df[(df['flex']==k) & (df['sex']=='female') & (df['employed']==1)]
    sns.distplot(tmp['hrwage'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=100, ax=ax[k])
#     ax[k].legend(['Flexibility Level ' + str(k)])
    ax[k].set_ylim([0,0.1])
    ax[k].set_xlim([0,75])
    if k == 1:
        ax[k].set(xlabel = 'Hourly Wage for Women with Flexible Schedule')
    elif k == 0:
        ax[k].set(xlabel = 'Hourly Wage for Women without Flexible Schedule')
    else:
        print("Not binary k")

# ax.set(xlabel="Distribution of Men's Hourly Wage (raw)")

plt.tight_layout()

fig.savefig('./hrwage_women_2flex.png', bbox_inches='tight', transparent=True)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8))

for k in range(2):
    tmp = men[(men['flex']==k) & (men['employed']==1)] #(df['sex']=='male') & 
    sns.distplot(tmp['wage_trunc'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=100, ax=ax[k])
#     ax[k].legend(['Flexibility Level ' + str(k)])
    ax[k].set_ylim([0,0.1])
    ax[k].set_xlim([0,75])
    if k == 1:
        ax[k].set(xlabel = 'Truncated Hourly Wage for Men with Flexible Schedule')
    elif k == 0:
        ax[k].set(xlabel = 'Truncated Hourly Wage for Men without Flexible Schedule')
    else:
        print("Not binary k")

# ax.set(xlabel="Distribution of Men's Hourly Wage (raw)")

plt.tight_layout()

fig.savefig('./wagetrunc_men_2flex.png', bbox_inches='tight', transparent=True)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8))

for k in range(2):
    tmp = women[(women['flex']==k) & (women['employed']==1)] #(df['sex']=='male') & 
    sns.distplot(tmp['wage_trunc2'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=100, ax=ax[k])
#     ax[k].legend(['Flexibility Level ' + str(k)])
    ax[k].set_ylim([0,0.1])
    ax[k].set_xlim([0,75])
    if k == 1:
        ax[k].set(xlabel = 'Truncated Hourly Wage for Women with Flexible Schedule')
    elif k == 0:
        ax[k].set(xlabel = 'Truncated Hourly Wage for Women without Flexible Schedule')
    else:
        print("Not binary k")

# ax.set(xlabel="Distribution of Men's Hourly Wage (raw)")

plt.tight_layout()

fig.savefig('./wagetrunc_women_2flex.png', bbox_inches='tight', transparent=True)

# Summary Statistics

In [ ]:
agg_dict_empl_3 = {
    'hrwage': ['count', 'min', 'mean', 'std'],
    'wage_trunc3': ['min', 'mean', 'std']
}

agg_dict_empl_2 = {
    'hrwage': ['count', 'min', 'mean', 'std'],
    'wage_trunc2': ['min', 'mean', 'std']
}

agg_dict_unempl = {
    'dur': ['count', 'min', 'max', 'mean', 'std']
}

agg_dict_pos = {
    'flex_sched_score': ['count'] #want to add percent!
}

### Men

In [ ]:
print(men.groupby(['flex_sched_score']).agg(agg_dict_empl_3).to_latex(float_format="%.2f"))

In [ ]:
print(men.groupby(['flex']).agg(agg_dict_empl_2).to_latex(float_format="%.2f"))

In [ ]:
print(men.agg(agg_dict_unempl).to_latex()) # by gender

### Women

In [ ]:
print(women.groupby(['flex_sched_score']).agg(agg_dict_empl_3).to_latex(float_format="%.2f"))

In [ ]:
print(women.groupby(['flex']).agg(agg_dict_empl_2).to_latex(float_format="%.2f"))

In [ ]:
print(women.agg(agg_dict_unempl).to_latex()) # by gender

In [ ]:
print(df.groupby(df['sex']).agg(agg_dict_unempl).to_latex(float_format="%.2f")) # by gender

In [ ]:
print(empl_df.groupby(['female']).agg(agg_dict_empl).to_latex()) # all

In [ ]:
print(unempl_df.agg(agg_dict_unempl).to_latex()) # by gender

# Scratch

## Before functionalizing

In [ ]:
# Estimate ζ, holding γ constant and cost = 0
params = np.array([λ, η, ζ, men['wage_trunc2'].mean(), men['wage_trunc2'].std()])

logL_opt = lambda x: log_L(men, 'flexsched', 'wage_trunc2', 'dur', 
                            np.array([0,0]), α, x[0], x[1], x[2], γ,
                            x[3], x[4])

est_zeta = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000, 'disp':True}, bounds=Bounds)
est_zeta.x


In [ ]:
# Estimate c, holding γ and ζ constant 
params = np.array([λ, η, c_k[1], men['wage_trunc2'].mean(), men['wage_trunc2'].std()])

logL_opt = lambda x: log_L(men, 'flexsched', 'wage_trunc2', 'dur', men['wage_trunc2'].min(), 
                            np.array([0,x[0]]), prob_k, α, x[1], x[2], est_zeta.x[2], est_gamma.x[2],
                            x[3], x[4])

est_c = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000, 'disp':True}, bounds=Bounds)
est_c.x


In [ ]:
## Estimate γ, holding ζ constant and cost = 0
# params = np.array([λ, η, γ, men['wage_trunc2'].mean(), men['wage_trunc2'].std()])

# Bootstrapping
bootstrapped_data = bootstrap(men, n_samples=30)

logL_gamma = []
lambdas_gamma = []
etas_gamma = []
mus_gamma = []
sigmas_gamma = []
gammas = []

for sample in bootstrapped_data:
    params = np.array([λ, η, γ, sample['wage_trunc2'].mean(), sample['wage_trunc2'].std()])
    
    logL_opt = lambda x: log_L(sample, 'flexsched', 'wage_trunc2', 'dur', sample['wage_trunc2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], ζ, x[2],
                            x[3], x[4])

    est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000}, bounds=Bounds)
    
    logL_gamma.append(est.fun)
    lambdas_gamma.append(est.x[0])
    etas_gamma.append(est.x[1])
    gammas.append(est.x[2])
    mus_gamma.append(est.x[3])
    sigmas_gamma.append(est.x[4])

In [ ]:
# Estimate ζ, holding γ constant and cost = 0
# params = np.array([λ, η, ζ, men['wage_trunc2'].mean(), men['wage_trunc2'].std()])

## Bootstrapping
bootstrapped_data = bootstrap(men, n_samples=30)

logL_zetas = []
lambdas_zetas = []
etas_zetas = []
mus_zetas = []
sigmas_zetas = []
zetas = []

for sample in bootstrapped_data:
    params = np.array([λ, η, ζ, sample['wage_trunc2'].mean(), sample['wage_trunc2'].std()])

    logL_opt = lambda x: log_L(sample, 'flexsched', 'wage_trunc2', 'dur', sample['wage_trunc2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], x[2], γ,
                            x[3], x[4])

    est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000}, bounds=Bounds)
    
    logL_zetas.append(est.fun)
    lambdas_zetas.append(est.x[0])
    etas_zetas.append(est.x[1])
    zetas.append(est.x[2])
    mus_zetas.append(est.x[3])
    sigmas_zetas.append(est.x[4])


#### Compare Bootstrapped Results

In [ ]:
mylist = [logL_gamma, lambdas_gamma, etas_gamma, mus_gamma, sigmas_gamma, gammas]

for i in mylist:
    fit_stats(i)

In [ ]:
mylist = [logL_zetas, lambdas_zetas, etas_zetas, mus_zetas, sigmas_zetas, zetas]

for i in mylist:
    fit_stats(i)

### Estimation: Men, Binary Location Flex

In [ ]:
# Estimation Specific Parameters

γ = 0.625 # Relative value of location flexibility to high salary from He et al (2021)

prob_k = np.array([0.584541, 0.415459])

In [ ]:
men['flexloc'].value_counts(normalize=True, sort=False)

In [ ]:
# Estimate γ, holding ζ constant cost = 0
params = np.array([λ, η, γ, men['wage_trunc_loc_2'].mean(), men['wage_trunc_loc_2'].std()])

logL_opt = lambda x: log_L(men, 'flexloc', 'wage_trunc_loc_2', 'dur', men['wage_trunc_loc_2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], ζ, x[2],
                            x[3], x[4])

est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000, 'disp':True}, bounds=Bounds)
est.x


In [ ]:
# Estimate ζ, holding γ constant and cost = 0
params = np.array([λ, η, ζ, men['wage_trunc_loc_2'].mean(), men['wage_trunc_loc_2'].std()])

logL_opt = lambda x: log_L(men, 'flexloc', 'wage_trunc_loc_2', 'dur', men['wage_trunc_loc_2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], x[2], γ,
                            x[3], x[4])

est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000, 'disp':True}, bounds=Bounds)
est.x


#### Bootstrapping

In [ ]:
## Estimate γ, holding ζ constant and cost = 0
bootstrapped_data = bootstrap(men, n_samples=30)

logL_gamma = []
lambdas_gamma = []
etas_gamma = []
mus_gamma = []
sigmas_gamma = []
gammas = []

for sample in bootstrapped_data:
    params = np.array([λ, η, γ, sample['wage_trunc_loc_2'].mean(), sample['wage_trunc_loc_2'].std()])
    
    logL_opt = lambda x: log_L(sample, 'flexloc', 'wage_trunc_loc_2', 'dur', sample['wage_trunc_loc_2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], ζ, x[2],
                            x[3], x[4])

    est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000}, bounds=Bounds)
    
    logL_gamma.append(est.fun)
    lambdas_gamma.append(est.x[0])
    etas_gamma.append(est.x[1])
    gammas.append(est.x[2])
    mus_gamma.append(est.x[3])
    sigmas_gamma.append(est.x[4])

In [ ]:
# Estimate ζ, holding γ constant and cost = 0
bootstrapped_data = bootstrap(men, n_samples=30)

logL_zetas = []
lambdas_zetas = []
etas_zetas = []
mus_zetas = []
sigmas_zetas = []
zetas = []

for sample in bootstrapped_data:
    params = np.array([λ, η, ζ, sample['wage_trunc_loc_2'].mean(), sample['wage_trunc_loc_2'].std()])

    logL_opt = lambda x: log_L(sample, 'flexloc', 'wage_trunc_loc_2', 'dur', sample['wage_trunc_loc_2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], x[2], γ,
                            x[3], x[4])

    est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000}, bounds=Bounds)
    
    logL_zetas.append(est.fun)
    lambdas_zetas.append(est.x[0])
    etas_zetas.append(est.x[1])
    zetas.append(est.x[2])
    mus_zetas.append(est.x[3])
    sigmas_zetas.append(est.x[4])


#### Compare Bootstrapped Results

In [ ]:
mylist = [logL_gamma, lambdas_gamma, etas_gamma, mus_gamma, sigmas_gamma, gammas]

for i in mylist:
    fit_stats(i)

In [ ]:
mylist = [logL_zetas, lambdas_zetas, etas_zetas, mus_zetas, sigmas_zetas, zetas]

for i in mylist:
    fit_stats(i)

In [ ]:
fig, ax = plt.subplots(3,2,figsize=(8,6))

sns.distplot(lambdas_gamma, ax=ax[0,0]).set(title="Lambda")
sns.distplot(etas_gamma, ax=ax[0,1]).set(title="Eta")
sns.distplot(mus_gamma, ax=ax[1,0]).set(title="Mu")
sns.distplot(sigmas_gamma, ax=ax[1,1]).set(title="Sigma")
sns.distplot(gammas, ax=ax[2,0]).set(title="Gammas")
sns.distplot(logL_gamma, ax=ax[2,1]).set(title="Log-Likelihood")

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(3,2,figsize=(8,6))

sns.distplot(lambdas_zetas, ax=ax[0,0]).set(title="Lambda")
sns.distplot(etas_zetas, ax=ax[0,1]).set(title="Eta")
sns.distplot(mus_zetas, ax=ax[1,0]).set(title="Mu")
sns.distplot(sigmas_zetas, ax=ax[1,1]).set(title="Sigma")
sns.distplot(zetas, ax=ax[2,0]).set(title="Zetas")
sns.distplot(logL_zetas, ax=ax[2,1]).set(title="Log-Likelihood")

plt.tight_layout()

### Estimation: Women, Binary Schedule Flex

In [ ]:
# Estimation Specific Parameters

γ = 0.6875 # Relative value of schedule flexibility to high salary from He et al (2021)

prob_k = np.array([0.40264, 0.59736])

In [ ]:
women['flexsched'].value_counts(normalize=True, sort=False)

In [ ]:
## Estimate γ, holding ζ constant and setting cost = 0
params = np.array([λ, η, γ, women['wage_trunc2'].mean(), women['wage_trunc2'].std()])

logL_opt = lambda x: log_L(women, 'flexsched', 'wage_trunc2', 'dur', women['wage_trunc2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], 1.28, x[2], 
                            x[3], x[4])

est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000, 'disp':True}, bounds=Bounds)
est.x


In [ ]:
## Estimate ζ, holding γ constant and setting cost=0
params = np.array([λ, η, ζ, women['wage_trunc2'].mean(), women['wage_trunc2'].std()])

logL_opt = lambda x: log_L(women, 'flexsched', 'wage_trunc2', 'dur', women['wage_trunc2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], x[2], -.144,
                            x[3], x[4])

est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000, 'disp':True}, bounds=Bounds)
est.x


#### Bootstrapping

In [ ]:
## Estimate γ, holding ζ constant and cost = 0

bootstrapped_data = bootstrap(women, n_samples=30)

logL_gamma = []
lambdas_gamma = []
etas_gamma = []
mus_gamma = []
sigmas_gamma = []
gammas = []

for sample in bootstrapped_data:
    params = np.array([λ, η, γ, sample['wage_trunc2'].mean(), sample['wage_trunc2'].std()])
    
    logL_opt = lambda x: log_L(sample, 'flexsched', 'wage_trunc2', 'dur', sample['wage_trunc2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], ζ, x[2],
                            x[3], x[4])

    est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000}, bounds=Bounds)
    
    logL_gamma.append(est.fun)
    lambdas_gamma.append(est.x[0])
    etas_gamma.append(est.x[1])
    gammas.append(est.x[2])
    mus_gamma.append(est.x[3])
    sigmas_gamma.append(est.x[4])

In [ ]:
# Estimate ζ, holding γ constant and cost = 0
bootstrapped_data = bootstrap(women, n_samples=30)

logL_zetas = []
lambdas_zetas = []
etas_zetas = []
mus_zetas = []
sigmas_zetas = []
zetas = []

for sample in bootstrapped_data:
    params = np.array([λ, η, ζ, sample['wage_trunc2'].mean(), sample['wage_trunc2'].std()])
    
    logL_opt = lambda x: log_L(sample, 'flexsched', 'wage_trunc2', 'dur', sample['wage_trunc2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], x[2], γ,
                            x[3], x[4])

    est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000}, bounds=Bounds)
    
    logL_zetas.append(est.fun)
    lambdas_zetas.append(est.x[0])
    etas_zetas.append(est.x[1])
    zetas.append(est.x[2])
    mus_zetas.append(est.x[3])
    sigmas_zetas.append(est.x[4])


#### Compare Bootstrapped Results

In [ ]:
mylist = [logL_gamma, lambdas_gamma, etas_gamma, mus_gamma, sigmas_gamma, gammas]

for i in mylist:
    fit_stats(i)

In [ ]:
mylist = [logL_zetas, lambdas_zetas, etas_zetas, mus_zetas, sigmas_zetas, zetas]

for i in mylist:
    fit_stats(i)

In [ ]:
fig, ax = plt.subplots(3,2,figsize=(8,6))

sns.distplot(lambdas_gamma, ax=ax[0,0]).set(title="Lambda")
sns.distplot(etas_gamma, ax=ax[0,1]).set(title="Eta")
sns.distplot(mus_gamma, ax=ax[1,0]).set(title="Mu")
sns.distplot(sigmas_gamma, ax=ax[1,1]).set(title="Sigma")
sns.distplot(gammas, ax=ax[2,0]).set(title="Gammas")
sns.distplot(logL_gamma, ax=ax[2,1]).set(title="Log-Likelihood")

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(3,2,figsize=(8,6))

sns.distplot(lambdas_zetas, ax=ax[0,0]).set(title="Lambda")
sns.distplot(etas_zetas, ax=ax[0,1]).set(title="Eta")
sns.distplot(mus_zetas, ax=ax[1,0]).set(title="Mu")
sns.distplot(sigmas_zetas, ax=ax[1,1]).set(title="Sigma")
sns.distplot(zetas, ax=ax[2,0]).set(title="Zetas")
sns.distplot(logL_zetas, ax=ax[2,1]).set(title="Log-Likelihood")

plt.tight_layout()

### Estimation: Women, Binary Location Flex

In [ ]:
women['flexloc'].value_counts(normalize=True, sort=False)

In [ ]:
# Utility measure for type of flex

γ = 0.625 # Relative value of location flexibility to high salary from He et al (2021)

prob_k = np.array([0.618812, 0.381188])


In [ ]:
## STRANGEST OUTPUT!

# Estimate γ, holding ζ constant and setting cost = 0
params = np.array([λ, η, γ, women['wage_trunc_loc_2'].mean(), women['wage_trunc_loc_2'].std()])

logL_opt = lambda x: log_L(women, 'flexloc', 'wage_trunc_loc_2', 'dur', women['wage_trunc_loc_2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], ζ, x[2], 
                            x[3], x[4])

est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000, 'disp':True}, bounds=Bounds)
est.x

In [ ]:
# Estimate ζ, holding γ constant and setting cost=0
params = np.array([λ, η, ζ, women['wage_trunc_loc_2'].mean(), women['wage_trunc_loc_2'].std()])

logL_opt = lambda x: log_L(women, 'flexloc', 'wage_trunc_loc_2', 'dur', women['wage_trunc_loc_2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], x[2], γ,
                            x[3], x[4])

est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000, 'disp':True}, bounds=Bounds)
est.x

#### Bootstrapping

In [ ]:
# Estimate γ, holding ζ constant and cost = 0

bootstrapped_data = bootstrap(women, n_samples=30)

logL_gamma = []
lambdas_gamma = []
etas_gamma = []
mus_gamma = []
sigmas_gamma = []
gammas = []

for sample in bootstrapped_data:
    params = np.array([λ, η, γ, sample['wage_trunc_loc_2'].mean(), sample['wage_trunc_loc_2'].std()])
    
    logL_opt = lambda x: log_L(sample, 'flexloc', 'wage_trunc_loc_2', 'dur', sample['wage_trunc_loc_2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], ζ, x[2],
                            x[3], x[4])

    est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000}, bounds=Bounds)
    
    logL_gamma.append(est.fun)
    lambdas_gamma.append(est.x[0])
    etas_gamma.append(est.x[1])
    gammas.append(est.x[2])
    mus_gamma.append(est.x[3])
    sigmas_gamma.append(est.x[4])

In [ ]:
# Estimate ζ, holding γ constant and cost = 0
bootstrapped_data = bootstrap(women, n_samples=30)

logL_zetas = []
lambdas_zetas = []
etas_zetas = []
mus_zetas = []
sigmas_zetas = []
zetas = []

for sample in bootstrapped_data:
    params = np.array([λ, η, ζ, sample['wage_trunc_loc_2'].mean(), sample['wage_trunc_loc_2'].std()])
    
    logL_opt = lambda x: log_L(sample, 'flexloc', 'wage_trunc_loc_2', 'dur', sample['wage_trunc_loc_2'].min(), 
                            np.array([0,0]), prob_k, α, x[0], x[1], x[2], γ,
                            x[3], x[4])

    est = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':1000}, bounds=Bounds)
    
    logL_zetas.append(est.fun)
    lambdas_zetas.append(est.x[0])
    etas_zetas.append(est.x[1])
    zetas.append(est.x[2])
    mus_zetas.append(est.x[3])
    sigmas_zetas.append(est.x[4])


#### Compare Bootstrapped Results

In [ ]:
mylist = [logL_gamma, lambdas_gamma, etas_gamma, mus_gamma, sigmas_gamma, gammas]

for i in mylist:
    fit_stats(i)

In [ ]:
mylist = [logL_zetas, lambdas_zetas, etas_zetas, mus_zetas, sigmas_zetas, zetas]

for i in mylist:
    fit_stats(i)

In [ ]:
fig, ax = plt.subplots(3,2,figsize=(8,6))

sns.distplot(lambdas_gamma, ax=ax[0,0]).set(title="Lambda")
sns.distplot(etas_gamma, ax=ax[0,1]).set(title="Eta")
sns.distplot(mus_gamma, ax=ax[1,0]).set(title="Mu")
sns.distplot(sigmas_gamma, ax=ax[1,1]).set(title="Sigma")
sns.distplot(gammas, ax=ax[2,0]).set(title="Gammas")
sns.distplot(logL_gamma, ax=ax[2,1]).set(title="Log-Likelihood")

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(3,2,figsize=(8,6))

sns.distplot(lambdas_zetas, ax=ax[0,0]).set(title="Lambda")
sns.distplot(etas_zetas, ax=ax[0,1]).set(title="Eta")
sns.distplot(mus_zetas, ax=ax[1,0]).set(title="Mu")
sns.distplot(sigmas_zetas, ax=ax[1,1]).set(title="Sigma")
sns.distplot(zetas, ax=ax[2,0]).set(title="Zetas")
sns.distplot(logL_zetas, ax=ax[2,1]).set(title="Log-Likelihood")

plt.tight_layout()

## Utility with wage and flex; Productivity with TFP coeff; Two-Stage Estimation

In [ ]:
# Two-stage estimation

## Labor Market Variables in the first stage
Bounds1 = ((0,999), (0,999), (0,999), (0,999))

params1 = np.array([λ, η, men['wage_trunc2'].mean(), men['wage_trunc2'].std()])

logL_opt1 = lambda x: log_L(men, 'flex', 'wage_trunc2', 'dur', men['wage_trunc2'].min(), 
                            np.array([0,7]), prob_k, α, x[0], x[1], ζ, γ,
                            x[2], x[3])

est2 = minimize(logL_opt1, params1, method='Nelder-Mead', options={'maxiter':500, 'disp':True}, bounds=Bounds1)

## Flexibility Variables in the second stage
params2 = np.array([c_k[1], ζ, γ])

logL_opt2 = lambda x: log_L(men, 'flex', 'wage_trunc2', 'dur', men['wage_trunc2'].min(), 
                            np.array([0,x[0]]), prob_k, α, est2.x[0], est2.x[1], x[1], x[2],
                            est2.x[2], est2.x[3])

est2_second = minimize(logL_opt2, params2, method='Nelder-Mead', options={'maxiter':500, 'disp':True})

In [ ]:
print("Men's Labor market variables [λ, η, μ, σ] = "+ str(est2.x))
print("Men's Flexibility variables [c(1), ζ, γ] = "+ str(est2_second.x))

In [ ]:
# Two-stage estimation with bootstrapping

## Labor Market Variables in the first stage: λ, η, μ, σ
Bounds1 = ((0,999), (0,999), (0,999), (0,999))
params1 = np.array([λ, η, men['wage_trunc2'].mean(), men['wage_trunc2'].std()])

## Flexibility Variables in the second stage: c(k), ζ, γ
params2 = np.array([c_k[1], ζ, γ])

## Bootstrapping
bootstrapped_data = bootstrap(men, n_samples=30)

logL1 = []
logL2 = []
lambdas = []
etas = []
mus = []
sigmas = []
cs = []
zetas = []
gammas = []

for sample in bootstrapped_data:
    logL_opt1 = lambda x: log_L(sample, 'flex', 'wage_trunc2', 'dur', sample['wage_trunc2'].min(), 
                            np.array([0,7]), prob_k, α, x[0], x[1], ζ, γ,
                            x[2], x[3])
    est2 = minimize(logL_opt1, params1, method='Nelder-Mead', bounds=Bounds1)#options={'maxiter':500, 'disp':True}, 
    
    logL_opt2 = lambda x: log_L(sample, 'flex', 'wage_trunc2', 'dur', sample['wage_trunc2'].min(), 
                            np.array([0,x[0]]), prob_k, α, est2.x[0], est2.x[1], x[1], x[2],
                            est2.x[2], est2.x[3])
    est2_second = minimize(logL_opt2, params2, method='Nelder-Mead')#, options={'maxiter':500, 'disp':True}
    
    logL1.append(est2.fun)
    logL2.append(est2_second.fun)
    lambdas.append(est2.x[0])
    etas.append(est2.x[1])
    mus.append(est2.x[2])
    sigmas.append(est2.x[3])
    cs.append(est2_second.x[0])
    zetas.append(est2_second.x[1])
    gammas.append(est2_second.x[2])

mylist = [logL1, logL2, lambdas, etas, mus, sigmas, cs, zetas, gammas]

for i in mylist:
    fit_stats(i)

In [ ]:
fig, ax = plt.subplots(3,3,figsize=(12,12))
sns.distplot(logL1, ax=ax[0,0]).set(title="Log-Likelihood of First Stage")
sns.distplot(logL2, ax=ax[0,1]).set(title="Log-Likelihood of Second Stage")
sns.distplot(lambdas, ax=ax[0,2]).set(title="Lambda")
sns.distplot(etas, ax=ax[1,0]).set(title="Eta")
sns.distplot(mus, ax=ax[1,1]).set(title="Mu")
sns.distplot(sigmas, ax=ax[1,2]).set(title="Sigma")
sns.distplot(cs, ax=ax[2,0]).set(title="Cost of Flexibility")
sns.distplot(zetas, ax=ax[2,1]).set(title="Zetas")
sns.distplot(gammas, ax=ax[2,2]).set(title="Gammas")

plt.tight_layout()

In [ ]:
# Binary flexibility

Bounds = ((0,999), (0,999), (0,999), (0,999))

params = np.array([λ, η, men['wage_trunc2'].mean(), men['wage_trunc2'].std()])

logL_opt = lambda x: log_L(men, 'flex', 'wage_trunc2', 'dur', men['wage_trunc2'].min(), 
                            np.array([0,10]), prob_k, α, x[0], x[1], ζ, γ,
                            x[2], x[3])

est2 = minimize(logL_opt, params, method='Nelder-Mead', bounds=Bounds, options={'maxiter':5000, 'disp':True})

In [ ]:
est2.x

In [ ]:
params = np.array([c_k[1], ζ, γ])

logL_opt = lambda x: log_L(men, 'flex', 'wage_trunc2', 'dur', men['wage_trunc2'].min(), 
                            np.array([0,x[0]]), prob_k, α, est2.x[0], est2.x[1], x[1], x[2],
                            est2.x[2], est2.x[3])

est2_second = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':500, 'disp':True})

In [ ]:
est2_second.x

In [ ]:
# Binary flexibility

Bounds = ((0,999), (0,999), (0,999), (0,999))

params = np.array([λ, η, women['wage_trunc2'].mean(), women['wage_trunc2'].std()])

logL_opt = lambda x: log_L(women, 'flex', 'wage_trunc2', 'dur', women['wage_trunc2'].min(), 
                            np.array([0,7]), prob_k, α, x[0], x[1], ζ, γ,
                            x[2], x[3])

est2 = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':500, 'disp':True}, bounds=Bounds)

In [ ]:
est2.x

In [ ]:
params = np.array([c_k[1], ζ, γ])

logL_opt = lambda x: log_L(women, 'flex', 'wage_trunc2', 'dur', women['wage_trunc2'].min(), 
                            np.array([0,x[0]]), prob_k, α, est2.x[0], est2.x[1], x[1], x[2],
                            est2.x[2], est2.x[3])

est2_second = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':500, 'disp':True})

In [ ]:
est2_second.x

In [ ]:
# Two-stage estimation

## Labor Market Variables in the first stage
Bounds1 = ((0,999), (0,999), (0,999), (0,999))

params1 = np.array([λ, η, women['wage_trunc2'].mean(), women['wage_trunc2'].std()])

logL_opt1 = lambda x: log_L(women, 'flex', 'wage_trunc2', 'dur', women['wage_trunc2'].min(), 
                            np.array([0,7]), prob_k, α, x[0], x[1], ζ, γ,
                            x[2], x[3])

est2 = minimize(logL_opt1, params1, method='Nelder-Mead', options={'maxiter':500, 'disp':True}, bounds=Bounds1)

## Flexibility Variables in the second stage
params2 = np.array([c_k[1], ζ, γ])

logL_opt2 = lambda x: log_L(women, 'flex', 'wage_trunc2', 'dur', women['wage_trunc2'].min(), 
                            np.array([0,x[0]]), prob_k, α, est2.x[0], est2.x[1], x[1], x[2],
                            est2.x[2], est2.x[3])

est2_second = minimize(logL_opt2, params2, method='Nelder-Mead', options={'maxiter':500, 'disp':True})

In [ ]:
print("Women's Labor market variables [λ, η, μ, σ] = "+ str(est2.x))
print("Women's Flexibility variables [c(1), ζ, γ] = "+ str(est2_second.x))

In [ ]:
params = np.array([c_k[1], c_k[2], λ, η, γ, men['wage_trunc3'].mean(), men['wage_trunc3'].std()])

logL_opt = lambda x: log_L(men, 'flex_sched_score', 'wage_trunc3', 'dur', men['wage_trunc3'].min(), 
                            np.array([0,x[0],x[1]]), prob_k, α, x[2], x[3], ζ, x[4],
                            x[5], x[6])

est4 = minimize(logL_opt, params, method='Nelder-Mead', options={'maxiter':800})

Runs if $\zeta = 1$ and $\gamma = 0$, as in the initial model, so it is a problem of using one flexibility marker to estimate 3 flexibility measures

In [ ]:
est4.success

In [ ]:
est4

In [ ]:
est4.fun

In [ ]:
# Parameters to be estimated 

c_k = np.array([0,5,10])
λ = 2.1
η = 2.1
μ = men['hrwage'].groupby(men['flex_sched_score']).mean().values
σ = men['hrwage'].groupby(men['flex_sched_score']).std().values

In [ ]:
logL_opt = lambda x: log_L(men, 'flex_sched_score', 'hrwage', 'dur', Uk, 
                            x[0], prob_k, 0.5, x[1], x[2],
                            x[3], x[4])

In [ ]:
params = np.array([c_k, float(λ), float(η), μ, σ])
params

In [ ]:
params[1]

In [ ]:
log_L(men, 'flex_sched_score', 'hrwage', 'dur', Uk, 
                            params[0], prob_k, 0.5, params[1], params[2],
                            params[3], params[4])

In [ ]:
logL_opt(params)

## Pr_wage_match, hazard, and logL with res_wage, mu, and sigma varying with k

In [ ]:
def Pr_wage_given_match(data: pd.DataFrame, flex: str, wage: str, res_wage: float, c_k: np.array, α: float, μ: float, σ: float):
# def Pr_wage_given_match(data: pd.DataFrame, flex: str, wage: str, res_wage: np.array, c_k: np.array, α: float, μ: np.array, σ: np.array):
    """
    Calculates probability of a wage draw conditional on a match being formed 
    
    Inputs
    - data: DataFrame
    - flex: string for name of flexibility column
    - wage: string for name of wage column
    - res_wage: Kx1 array of observed minimum wages for each flexibility level
    - c_k: Kx1 array of cost of providing flexibility
    - α: bargaining parameter
    - μ: array of location parameter of the log-normal wage distribution for each flexibility level
    - σ: array of scale parameter of the log-normal wage distribution for each flexibility level
    
    Functions
    - lognormpdf(x: np.array, μ: float, σ: float)
    - lognormsf(x: np.array, μ: float, σ: float)
    """
    employed_indiv = np.zeros(1) #sets first entry to zero 
# With U, μ and σ constant in flex level k
    for k in range(len(c_k)):
        tmp = data[data[flex]==k]
        g = ( 1/( α*(k+1) ) ) * lognormpdf( ( 1/( α*(k+1) ) )*( tmp[wage] - (1-α)*res_wage + α*c_k[k] ), μ, σ )
        G_tilde = lognormsf( ( 1/(k+1) )*( res_wage + c_k[k] ), μ, σ )
        divide_thing = g/G_tilde
        employed_indiv = np.append(employed_indiv, divide_thing)
# # With U, μ and σ varying with flex level k - unidentified    
#     for k in range(len(res_wage)):
#         tmp = data[data[flex]==k]
#         g = ( 1/( α*(k+1) ) ) * lognormpdf( ( 1/( α*(k+1) ) )*( tmp[wage] - (1-α)*res_wage[k] + α*c_k[k] ), μ[k], σ[k] )
#         G_tilde = lognormsf( ( 1/(k+1) )*( res_wage[k] + c_k[k] ), μ[k], σ[k] )
#         divide_thing = g/G_tilde
#         employed_indiv = np.append(employed_indiv, divide_thing)
    
    return employed_indiv[1:] #removes first entry 

In [ ]:
def hazard(res_wage: float, c_k: np.array, p_k: np.array, λ: float, μ: float, σ: float):
# def hazard(res_wage: np.array, c_k: np.array, p_k: np.array, λ: float, μ: np.array, σ: np.array):
    """
    Calculates the hazard rate out of employment 
    
    Inputs
    - res_wage: Kx1 array of observed minimum wages for each flexibility level
    - c_k: Kx1 array of cost of providing flexibility
    - p_k: Kx1 array of probability of each level of flexibility
    - λ: arrival rate of offer
    - μ: array of location parameter of the log-normal wage distribution for each flexibility level
    - σ: array of scale parameter of the log-normal wage distribution for each flexibility level
    
    Functions
    - lognormsf(x: np.array, μ: float, σ: float)
    """
    
    prob_sum = 0
    
#     if len(res_wage)!=len(c_k):
#         return print("Length of res_wage and c_k do not match.")
#     elif len(res_wage)!=len(p_k):
#         return print("Length of res_wage and p_k do not match.")
    if len(p_k)!=len(c_k):
        return print("Length of p_k and c_k do not match.")
    else:
# With U, μ and σ constant in flex level k
        for k in range(len(c_k)):
            prob_sum += p_k[k] * lognormsf( ( 1/(k+1) )*( res_wage + c_k[k]), μ, σ ) #k+1 because Python index 0

# # With U, μ and σ varying with flex level k - unidentified    
#         for k in range(len(res_wage)):
#             prob_sum += p_k[k] * lognormsf( ( 1/(k+1) )*( res_wage[k] + c_k[k]), μ[k], σ[k] ) #k+1 because Python index 0
    
    return λ*prob_sum#[0]

In [ ]:
def log_L(data: pd.DataFrame, flex: str, wage: str, dur: str, res_wage: float, c_k: np.array, p_k: np.array, α: float, λ: float, η: float, μ: float, σ: float):
# def log_L(data: pd.DataFrame, flex: str, wage: str, dur: str, res_wage: np.array, c_k: np.array, p_k: np.array, α: float, λ: float, η: float, μ: np.array, σ: np.array):
    """
    
    Inputs
    - data: DataFrame of all individuals
    - flex: string for column of flexibility index (k)
    - wage: string for column of wage data 
    - dur: string for unemployment duration data
    - res_wage: Kx1 array of observed minimum wages for each flexibility level
    - c_k: Kx1 array of cost of providing flexibility
    - p_k: Kx1 array of probability of each level of flexibility
    - α: bargaining parameter
    - λ: arrival rate of offer
    - η: termination rate
    - μ: array of location parameter of the log-normal wage distribution for each flexibility level
    - σ: array of scale parameter of the log-normal wage distribution for each flexibility level
    
    Functions
    - hazard(res_wage: np.array, c_k: np.array, p_k: np.array, λ: float, μ: float, σ: float)
    - Pr_wage_given_match(data: pd.DataFrame, flex: str, wage: str, res_wage: np.array, c_k: np.array,  α: float, μ: float, σ: float)
    """
    
    N_log_h = data.count() * np.log( hazard(res_wage, c_k, p_k, λ, μ, σ) )
    N_log_h_plus_η = data.count() * np.log( hazard(res_wage, c_k, p_k, λ, μ, σ) + η )
    
    empl_data = np.sum( np.log( Pr_wage_given_match(data, flex, wage, res_wage, c_k, α, μ, σ) ) )
    
    Nu_log_η = data[dur].count() * np.log(η)
    
    unempl_data = hazard(res_wage, c_k, p_k, λ, μ, σ) * np.sum(data[dur])
    
    logL = -(N_log_h - N_log_h_plus_η + empl_data + Nu_log_η - unempl_data)
    
    return logL[0]

## Old Hazard and Log L (did not copy Pr_wage_match in time)

In [ ]:
def hazard(res_wage: np.array, c_k: np.array, p_k: np.array, λ: float, μ: np.array, σ: np.array):
    """
    Calculates the hazard rate out of employment 
    
    Inputs
    - res_wage: Kx1 array of observed minimum wages for each flexibility level
    - c_k: Kx1 array of cost of providing flexibility
    - p_k: Kx1 array of probability of each level of flexibility
    - λ: arrival rate of offer
    - μ: array of location parameter of the log-normal wage distribution for each flexibility level
    - σ: array of scale parameter of the log-normal wage distribution for each flexibility level
    
    Functions
    - lognormsf(x: np.array, μ: float, σ: float)
    """
    
    prob_sum = 0
    
    if len(res_wage)!=len(c_k):
        return print("Length of res_wage and c_k do not match.")
    elif len(res_wage)!=len(p_k):
        return print("Length of res_wage and p_k do not match.")
    elif len(p_k)!=len(c_k):
        return print("Length of p_k and c_k do not match.")
    else:
        for k in range(len(res_wage)):
            prob_sum += p_k[k] * lognormsf( ( 1/(k+1) )*( res_wage[k] + c_k[k]), μ[k], σ[k] ) #k+1 because Python index 0
    
    return λ*prob_sum

In [ ]:
def log_L(wage: np.array, k: np.array, res_wage: np.array, c_k: np.array, p_k: np.array, dur: np.array, α: float, λ: float, η: float, μ: np.array, σ: np.array):
    """
    
    Inputs
    - wage: Ne x 1 array of observed wage data 
    - k: Ne x 1 array of observed flexibility level data
    - res_wage: Kx1 array of observed minimum wages for each flexibility level
    - c_k: Kx1 array of cost of providing flexibility
    - p_k: Kx1 array of probability of each level of flexibility
    - dur: Nu x 1 array of observed unemployment duration data
    - α: bargaining parameter
    - λ: arrival rate of offer
    - η: termination rate
    - μ: array of location parameter of the log-normal wage distribution for each flexibility level
    - σ: array of scale parameter of the log-normal wage distribution for each flexibility level
    
    Functions
    - hazard(res_wage: np.array, c_k: np.array, p_k: np.array, λ: float, μ: float, σ: float)
    - Pr_wage_given_match(wage: np.array, k: np.array, res_wage: np.array, c_k: np.array,  α: float, μ: float, σ: float)
    """
    
    N_log_h = len(wage) * np.log( hazard(res_wage, c_k, p_k, λ, μ, σ) )
    N_log_h_plus_η = len(wage) * np.log( hazard(res_wage, c_k, p_k, λ, μ, σ) + η )
    
    empl_data = np.sum( np.log( Pr_wage_given_match(wage, k, res_wage, c_k,  α, μ, σ) ) )
    
    Nu_log_η = len(dur) * np.log(η)
    
    unempl_data = hazard(res_wage, c_k, p_k, λ, μ, σ) * np.sum(dur)
    
    logL = N_log_h - N_log_h_plus_η + empl_data + Nu_log_η - unempl_data
    
    return logL

In [ ]:
empl_men = men[men['employed']==1]
len(empl_men)

In [ ]:
unempl_men = men[men['employed']==0]
len(unempl_men)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(12, 8))

for k in range(3):
    tmp = df[(df['flex_sched_score']==k) & (df['sex']=='female') & (df['employed']==1)]
    sns.distplot(tmp['hrwage'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=100, ax=ax[k])
    ax[k].legend(['Flex Level ' + str(k)])
    ax[k].set_ylim([0,0.1])
    ax[k].set_xlim([0,75])


#ax.set(xlabel="Distribution of Men's Hourly Wage")

plt.tight_layout()

# fig.savefig('./figures/wage_noMin.png', bbox_inches='tight', transparent=True)